# Hyperameter tuning (of additional layer(s) added during transfer learning, with experimental training set) using Keras tuner
### target is validation loss, best model is saved, tunable parameters are number of hidden layers, number of units in each layer, batch size, 

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import math
from LSTMutils import MeanVarianceLogLikelyhoodLoss
from LSTMutils import LSTMnetwork
from sklearn.model_selection import train_test_split
import keras_tuner as kt

np.random.seed(42)
tf.random.set_seed(42) 

SequenceLength = 250
validation_split = 0.25
# batch_size = 32
NumEpochs = 2000
test_split = 0.2

reconstructed_model = keras.models.load_model("../Models/SimulatedDataPretrainedModel",custom_objects={"MeanVarianceLogLikelyhoodLoss": MeanVarianceLogLikelyhoodLoss})

reconstructed_model.trainable = False

# read experimental data and split into time (s) converted to hours, concentration labels and time series data
df_full = pd.read_csv(r"../TrainingData/ExperimentalTrainingSet.csv",sep=',',header=None)
time = df_full.iloc[0,1:]/3600
labels = df_full.iloc[1:,0]
df_data = df_full.iloc[1:,1:]

df = pd.read_csv(r"../TrainingData/SimulatedTrainingSet10000.csv",sep=',',header=0)
df_simulated = df.iloc[:,1:]

# split data into stratified train and test sets, size defined by the test_split variable
# the split will always be the same provided the data is in the same order, the same random_state is used,
# and strangely the labels used for stratification are always the same type (str)

df_train, df_test = train_test_split(df_data, test_size=test_split, train_size=1-test_split, random_state=42, shuffle=True, stratify=labels)

train_labels = df_train.iloc[:,0]

df_train, df_val = train_test_split(df_train, test_size=validation_split, train_size=1-validation_split, random_state=42, shuffle=True, stratify=train_labels)

# normalise time series data
min_value, max_value = df_train.min().min(), df_train.max().max()
df_norm_train = (df_train - min_value)/(max_value - min_value)
df_norm_test = (df_test - min_value)/(max_value - min_value)
df_norm_val = (df_val - min_value)/(max_value - min_value)
    
X_train = df_norm_train.iloc[:,:SequenceLength].values
y_train = df_norm_train.iloc[:,SequenceLength-1].values
X_train = np.expand_dims(X_train, 2)
y_train = np.broadcast_to(y_train[:,None], (y_train.shape[0],SequenceLength))
y_train = np.expand_dims(y_train, 2)

X_val = df_norm_val.iloc[:,:SequenceLength].values
y_val = df_norm_val.iloc[:,SequenceLength-1].values
X_val = np.expand_dims(X_val, 2)
y_val = np.broadcast_to(y_val[:,None], (y_val.shape[0],SequenceLength))
y_val = np.expand_dims(y_val, 2)


#Implementation informed by https://github.com/keras-team/keras-tuner/issues/122

class LSTMHyperModel(kt.HyperModel):
    
    def build(self,hp):
        model = keras.models.Sequential([reconstructed_model
                        , keras.layers.LSTM(hp.Choice("layer_1_units", [2,5,10,20,50,100,200,500]), return_sequences=True)])#, kernel_regularizer=keras.regularizers.L2(hp.Choice("L2Regularizer1", [0., 1e-4]))))
#         if hp.Boolean("layer_1_dropout"):
#             model.add(keras.layers.Dropout(rate=0.25))
        for i in range(hp.Int("LSTM Layers", min_value = 0, max_value = 8)):
             model.add(keras.layers.LSTM(hp.Choice(f"layer_{i+2}_units", [2,5,10,20,50,100,200,500]), return_sequences=True))#, kernel_regularizer=keras.regularizers.L2(hp.Choice(f"L2Regularizer{i+2}", [0., 1e-4]))))
# #             if hp.Boolean(f"layer_{i+2}_Dropout"):
# #                 model.add(keras.layers.Dropout(rate=0.25))
        #learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")
        model.add(keras.layers.LSTM(2, activation='softplus',return_sequences=True))

        model.compile(optimizer="adam",loss = MeanVarianceLogLikelyhoodLoss)
    
        return model

    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,batch_size=hp.Choice("batch_size",[16,32,64,128,254]),**kwargs)
    
tuner = kt.RandomSearch(#kt.BayesianOptimization(#kt.Hyperband(#
    LSTMHyperModel(),
    objective='val_loss',
    overwrite = True,#False,#
    #beta=5,
    #factor = 3,
    #hyperband_iterations = 1,
    #max_epochs = 100,
    max_trials = 300,
    seed = 42,
    directory = "../Models/ExperimentalDataTransferLearningModelTuner")
        
tuner.search(X_train, y_train, epochs=NumEpochs, validation_data=(X_val,y_val), callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',patience=100)])
best_model = tuner.get_best_models()[0]
best_model.summary()
best_model.save("../Models/ExperimentalDataTransferLearningModelTuner")

Trial 69 Complete [00h 00m 43s]
val_loss: nan

Best val_loss So Far: -3.4095356464385986
Total elapsed time: 15h 24m 56s

Search: Running Trial #70

Value             |Best Value So Far |Hyperparameter
10                |5                 |layer_1_units
2                 |0                 |LSTM Layers
64                |16                |batch_size
2                 |None              |layer_2_units
10                |None              |layer_3_units
2                 |None              |layer_4_units
50                |None              |layer_5_units
5                 |None              |layer_6_units
5                 |None              |layer_7_units
2                 |None              |layer_8_units
200               |None              |layer_9_units

Epoch 1/2000
4/4 [==============================] - 7s 706ms/step - loss: 10.3235 - val_loss: 9.9169
Epoch 2/2000
4/4 [==============================] - 1s 271ms/step - loss: 9.6629 - val_loss: 9.1488
Epoch 3/2000
4/4 [===========

Epoch 73/2000
4/4 [==============================] - 1s 276ms/step - loss: 0.1229 - val_loss: 0.1319
Epoch 74/2000
4/4 [==============================] - 1s 272ms/step - loss: 0.1229 - val_loss: 0.1319
Epoch 75/2000
4/4 [==============================] - 1s 276ms/step - loss: 0.1229 - val_loss: 0.1319
Epoch 76/2000
4/4 [==============================] - 1s 269ms/step - loss: 0.1229 - val_loss: 0.1319
Epoch 77/2000
4/4 [==============================] - 1s 273ms/step - loss: 0.1229 - val_loss: 0.1319
Epoch 78/2000
4/4 [==============================] - 1s 277ms/step - loss: 0.1229 - val_loss: 0.1319
Epoch 79/2000
4/4 [==============================] - 1s 273ms/step - loss: 0.1229 - val_loss: 0.1319
Epoch 80/2000
4/4 [==============================] - 1s 260ms/step - loss: 0.1229 - val_loss: 0.1319
Epoch 81/2000
4/4 [==============================] - 1s 267ms/step - loss: 0.1229 - val_loss: 0.1319
Epoch 82/2000
4/4 [==============================] - 1s 267ms/step - loss: 0.1229 - val_los

KeyboardInterrupt: 

In [3]:
tuner.results_summary(20)

Results summary
Results in ../Models/ExperimentalDataTransferLearningModelTuner/untitled_project
Showing 20 best trials
Trial summary
Hyperparameters:
layer_1_units: 5
LSTM Layers: 0
batch_size: 32
Score: -3.4169135093688965
Trial summary
Hyperparameters:
layer_1_units: 2
LSTM Layers: 0
batch_size: 32
layer_2_units: 2
layer_3_units: 500
layer_4_units: 500
layer_5_units: 2
layer_6_units: 5
layer_7_units: 2
layer_8_units: 500
layer_9_units: 500
Score: -3.3951098918914795
Trial summary
Hyperparameters:
layer_1_units: 2
LSTM Layers: 0
batch_size: 32
layer_2_units: 2
layer_3_units: 2
layer_4_units: 500
layer_5_units: 500
layer_6_units: 2
layer_7_units: 2
layer_8_units: 500
layer_9_units: 500
Score: -3.357745885848999
Trial summary
Hyperparameters:
layer_1_units: 2
LSTM Layers: 0
batch_size: 32
layer_2_units: 2
layer_3_units: 500
Score: -3.3499293327331543
Trial summary
Hyperparameters:
layer_1_units: 2
LSTM Layers: 0
batch_size: 32
layer_2_units: 2
layer_3_units: 2
layer_4_units: 2
layer_5_